In [ ]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import openpyxl  # Asegúrate de importar openpyxl aquí

# Ruta del archivo Excel de entrada
input_excel_path = "C:\\Users\\constanza.perez\\Downloads\\CONSOLIDADO_LEAD_NUEVO_FINAL_FINAL.xlsx"
# Nombre de la hoja de Excel de entrada
input_sheet_name = "ID"
# Ruta base del archivo Excel de salida
output_excel_path = "C:\\Users\\constanza.perez\\Downloads\\RESULTADO.xlsx"
# Nombre de la hoja de Excel de salida
output_sheet_name = "Hoja2"

# Leer el archivo Excel de entrada
df = pd.read_excel(input_excel_path, sheet_name=input_sheet_name)

# Iterar sobre cada fila del dataframe
for index, row in df.iterrows():
    # Inicializar el navegador Chrome
    driver = webdriver.Chrome()
    # Abrir la página web
    driver.get("https://www.spensiones.cl/apps/certificados/formConsultaAfiliacion.php")

    # Tomar el dato de la columna 'nombre_columna' de la fila actual
    rut = row['RUT']

    try:
        # Buscar el cuadro de texto por su nombre
        rut_input = driver.find_element(By.NAME, 'rut')
        rut_input.click()

        # Insertar el RUT en el cuadro de texto
        rut_input.send_keys(Keys.CONTROL + "a")  # Seleccionar todo el texto en el cuadro
        rut_input.send_keys(Keys.DELETE)  # Borrar el texto existente
        rut_input.send_keys(str(rut))  # Ingresar el RUT sin puntos ni guión

        # Hacer clic en el botón "Buscar"
        buscar_button = driver.find_element(By.ID, 'btn_buscar')
        buscar_button.click()

        # Esperar a que el resultado sea visible en la página
        wait = WebDriverWait(driver, 20)
        resultado_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[contains(text(), "Certifico que")]')))
        resultado = resultado_element.text
    except NoSuchElementException:
        print(f"No se encontró el elemento 'rut' en la página para el RUT {rut}.")
        continue
    except TimeoutException:
        print(f"No se pudo encontrar el texto para el RUT {rut}. Continuando con el siguiente RUT.")

    # Cargar el archivo Excel de salida
    wb = openpyxl.load_workbook(output_excel_path)
    sheet = wb[output_sheet_name]

    # Obtener la última fila utilizada
    last_row = sheet.max_row + 1

    # Escribir el resultado en la siguiente fila vacía
    sheet.cell(row=last_row, column=1, value=rut)
    sheet.cell(row=last_row, column=2, value=resultado)

    # Guardar los cambios
    wb.save(output_excel_path)

    # Cerrar el navegador
    driver.quit()